In [1]:
import os
import sys

MODULE_PATH = 'C:\Github\DL_Study\CNN'

sys.path.insert(0, MODULE_PATH)

In [2]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy
import time

from ShuffleNet import *

# set random seed
torch.manual_seed(42)

In [3]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

def np_to_tensor(data):
    device = get_device()
    return torch.tensor(data).float().to(device)

# configuration setting
def model_config():
    # parameter for CNN Model
    epochs = [30]
    batch_size = [32]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config, verbose=0):

    # unpack config
    n_epochs, n_batch, learning_rate = config
    # use ShuffleNet for CNN
    model = ShuffleNet(groups=3, in_channels=1)
    if torch.cuda.is_available():
        model.cuda()

    # define Loss and Optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    data_size = train_X.size(0)
    max_iters = data_size//n_batch

    for epoch in range(1, n_epochs+1):
        #shuffle data
        idx = numpy.random.permutation(numpy.arange(data_size))
        x_data = train_X[idx]
        y_data = train_y[idx]

        epoch_loss = 0
        start_time = time.time()
        for it in range(max_iters):
            batch_x = x_data[it*n_batch:(it+1)*n_batch]
            batch_y = y_data[it*n_batch:(it+1)*n_batch]

            optimizer.zero_grad()
            predict = model(batch_x)
            loss = criterion(predict, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss+= loss.item()
        avg_loss = epoch_loss/max_iters

        if verbose:
            duration = start_time-time.time()
            print(f'epoch:{epoch}/{epochs}, ì‹œê°„:{duration:.2f}[s], loss:{avg_loss:.5f}')


    return model

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [4]:
import pandas as pd
import numpy as np
import numpy
import time
from datetime import datetime

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00247/'
data_name = 'data_akbilgic.xlsx'
df = pd.read_excel(data_url+data_name, header=1)
df.drop(columns=df.columns[[0]], axis=1, inplace=True)
cols = df.columns.tolist()
cols = cols[3:]+cols[:3]
df = df[cols]
df.head()

C:\Users\lab\AppData\Local\Temp/ipykernel_16224/4288721627.py:12: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel(data_url+data_name, header=1)


,DAX,FTSE,NIKKEI,BOVESPA,EU,EM,ISE,ISE.1,SP
0,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376,-0.004679
1,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,0.007787
2,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469
3,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391
4,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533


In [5]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 9
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var1(t-23),...,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var1(t)
24,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376,-0.004679,0.008455,...,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.016233,0.029306
25,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,0.007787,-0.017833,...,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.026541,0.004766
26,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469,-0.011726,...,0.004766,0.003651,-0.013411,-0.015462,0.005627,0.007895,0.011353,0.021468,0.001484,-0.035170
27,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391,-0.019873,...,-0.035170,-0.022182,-0.002902,-0.021440,-0.024388,-0.002139,-0.040542,-0.043907,-0.050369,0.005434
28,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533,-0.013526,...,0.005434,0.005019,-0.030745,-0.008799,0.001097,-0.007926,-0.022106,-0.033893,0.007923,-0.027421


In [6]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []

i = 1

print('config : epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    train_X = np_to_tensor(train_X)
    train_y = np_to_tensor(train_y)
    val_X = np_to_tensor(val_X)

    train_cv_X = np_to_tensor(train_cv_X)
    train_cv_y = np_to_tensor(train_cv_y)
    test_cv_X = np_to_tensor(test_cv_X)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        output = model(val_X)
        # for prevent cuda memory out
        predicted = output.data.cpu().numpy()
        error = MSE_metric(predicted, val_y)   # mse
        print(f'error(rmse):{np.sqrt(error):.5f}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)
    history.append(errors)

    # outer
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check time
    duration = time.time() - start_time
    output = selected_model(test_cv_X)
    predicted = output.data.cpu().numpy()
    
    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.size(0)}, val-size:{val_X.size(0)}, test-size:{test_cv_X.size(0)}')
    print(f'best_model => error(rmse) : {rmse:.5f}, param:{configs[param]}, times: {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 32, 0.01] model ==  

C:\Users\lab\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


error(rmse):0.15007
 == train [30, 32, 0.001] model ==  error(rmse):0.03251
train-size:55, val-size:13, test-size:42
best_model => error(rmse) : 0.04779, param:[30, 32, 0.001], times: 2.442

fold : 2/10
 == train [30, 32, 0.01] model ==  error(rmse):0.09673
 == train [30, 32, 0.001] model ==  error(rmse):0.03864
train-size:88, val-size:22, test-size:42
best_model => error(rmse) : 0.02961, param:[30, 32, 0.001], times: 3.666

fold : 3/10
 == train [30, 32, 0.01] model ==  error(rmse):0.02295
 == train [30, 32, 0.001] model ==  error(rmse):0.02551
train-size:122, val-size:30, test-size:42
best_model => error(rmse) : 0.02831, param:[30, 32, 0.01], times: 4.880

fold : 4/10
 == train [30, 32, 0.01] model ==  error(rmse):0.03927
 == train [30, 32, 0.001] model ==  error(rmse):0.01931
train-size:156, val-size:38, test-size:42
best_model => error(rmse) : 0.01722, param:[30, 32, 0.001], times: 7.361

fold : 5/10
 == train [30, 32, 0.01] model ==  error(rmse):0.01902
 == train [30, 32, 0.001] m

In [7]:
def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=0.0007047551140983898, std=0.0005832339272529215
RMSE: mean=0.024773951310683426, std=0.009539730109089916
MAE: mean=0.019606560952633682, std=0.0077329030447378655

[training time]
mean : 9.271704697608948, last:15.973591327667236
